# CSE 6240 - Web Search & Text Mining - Homework 3

In [1]:
# __author__ = 'Bhanu Verma'
# GTid = '903151012'

In [2]:
import csv
import pprint
import math
import numpy as np
from scipy.optimize import fmin_bfgs

In [3]:
def read_data(path):
    x = []
    y = []
    with open(path, 'r') as datafile:
        file_reader = csv.reader(datafile, delimiter=',')
        
        for row in file_reader:
            x.append((float(row[0]),float(row[1])))
            y.append(int(row[2]))
            
    return x, y

In [4]:
def get_sigmoid(theta, train_x):
    theta = np.reshape(theta,(len(theta),1))        # convert it to a column vector first
    z = np.dot(train_x,theta)                       # do matrix multiplication of the vectors
    den = 1.0 + math.e ** (-1.0 * z)                # calculate the denominator for the sigmoid
    sigmoid_val = 1.0/den
    return sigmoid_val

In [5]:
def get_h_theta(theta, train_x):
    h_theta_val = get_sigmoid(theta, train_x)
    return h_theta_val

In [6]:
def cost_function(theta, train_x, train_y, lambda_val = 0.0):
    training_size = float(train_x.shape[0])
    
    term_a = (1.0 * train_y) * np.log(get_h_theta(theta, train_x))
    term_b = (1.0 - train_y) * np.log(1.0 - get_h_theta(theta, train_x))
    cost = -(term_a + term_b)
    cost += (float(lambda_val) / (2.0 * training_size)) * np.sum(theta[1:]**2)
    
    for i in range(cost.size):
        if np.isnan(cost[i][0]):
            cost[i][0] = 0
        elif (cost[i][0] == np.inf):
            cost[i][0] = 10000
    
    cost = np.sum(cost) / training_size
    
    return cost

In [7]:
def gradient_function(theta, train_x, train_y, lambda_val):
    training_size = float(train_x.shape[0])
    delta_val = np.sum((get_h_theta(theta, train_x) - train_y) * train_x, axis=0)
    grad = (1.0 / training_size) * delta_val
    grad[1:] += (float(lambda_val) / training_size) * theta[1:]
    
    return grad

In [8]:
def run_optimization(train_x, train_y, lambda_val):
    training_size, dim_size = train_x.shape
    train_X = np.concatenate((np.ones((training_size, 1), dtype=float), train_x), axis=1)
    train_Y = train_y.reshape((training_size, 1))
    dim_size += 1                  # adjusting for theta zero
    theta = np.random.rand(dim_size, 1)
    optimized_theta = fmin_bfgs(cost_function, theta, fprime=gradient_function, args=(train_X,train_Y,lambda_val))
    
    return optimized_theta

In [9]:
filePath = 'data/ex2data1.txt'
x, y = read_data(filePath)
train_x = np.array(x)
train_y = np.array(y).reshape((len(y),1))

In [10]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
optimized_theta = run_optimization(train_x, train_y, 1)

In [ ]:
def make_prediction(opt_theta, train_x_item):
    train_X_item = np.concatenate((np.ones(1, dtype=float), train_x_item))
    val = get_h_theta(opt_theta, train_X_item)
    
    if val > 0.5:
        return 1
    else:
        return 0

In [ ]:
def get_accuracy(optimized_theta, train_x):
    ind = 0
    success_count = 0
    for row in train_x:
        pred_val = make_prediction(optimized_theta,row)
        if pred_val == train_y[ind]:
            success_count += 1
        ind += 1

    accuracy = success_count/(float(train_y.shape[0]))

In [ ]:
accuracy_score = get_accuracy(optimized_theta, train_x)
print("Accuracy: ", accuracy_score)